##Ejercicio 1

In [9]:
import random
import csv
import os

class Paciente:

    def __init__(self, id, nombre, edad, saturacion, vacunas, prueba_covid=None):
        self.id = id
        self.nombre = nombre
        self.edad = edad
        self.saturacion = saturacion
        self.vacunas = vacunas
        self.prueba_covid = prueba_covid
        self.destino = None

    def __str__(self):
        return f"ID: {self.id}, Nombre: {self.nombre}, Edad: {self.edad}, Saturación: {self.saturacion}, Vacunas: {self.vacunas}, Prueba COVID: {self.prueba_covid}, Destino: {self.destino}"

    def determinar_destino(self, camillas_disponibles):

        if self.saturacion < 90:
            self.destino = "Sala COVID"
            return self.destino


        if self.vacunas < 2:
            self.destino = "Sala COVID"
            return self.destino


        if self.prueba_covid is True:
            self.destino = "Sala COVID"
            return self.destino


        if camillas_disponibles > 0:
            self.destino = "Urgencias"
            return self.destino
        else:
            self.destino = "Espera"
            return self.destino


class GeneradorDatos:

    def __init__(self):
        self.nombres = ["Juan", "María", "Pedro", "Ana", "Carlos", "Laura", "Luis", "Sofía",
                       "Miguel", "Elena", "José", "Carmen", "Antonio", "Isabel", "David"]
        self.apellidos = ["García", "López", "Martínez", "Rodríguez", "González", "Fernández",
                         "Pérez", "Sánchez", "Ramírez", "Torres", "Díaz", "Vásquez", "Romero"]

    def generar_nombre(self):
        nombre = random.choice(self.nombres)
        apellido = random.choice(self.apellidos)
        return f"{nombre} {apellido}"

    def generar_pacientes(self, cantidad):
        pacientes = []
        for i in range(1, cantidad + 1):
            nombre = self.generar_nombre()
            edad = random.randint(18, 85)


            saturacion = random.randint(85, 99)
            vacunas = random.randint(0, 3)

            prueba_covid = None

            if saturacion >= 90 and vacunas >= 2:
                prueba_covid = random.choice([True, False])

            paciente = Paciente(i, nombre, edad, saturacion, vacunas, prueba_covid)
            pacientes.append(paciente)

        return pacientes


class GestorArchivos:

    @staticmethod
    def escribir_archivo_entrada(pacientes, nombre_archivo):
        with open(nombre_archivo, 'w', newline='') as archivo:
            escritor = csv.writer(archivo)
            escritor.writerow(['ID', 'Nombre', 'Edad', 'Saturacion', 'Vacunas', 'Prueba_COVID'])

            for paciente in pacientes:

                prueba_covid_str = "" if paciente.prueba_covid is None else str(paciente.prueba_covid)
                escritor.writerow([paciente.id, paciente.nombre, paciente.edad,
                                  paciente.saturacion, paciente.vacunas, prueba_covid_str])

    @staticmethod
    def leer_archivo_entrada(nombre_archivo):
        pacientes = []
        with open(nombre_archivo, 'r') as archivo:
            lector = csv.reader(archivo)
            next(lector)

            for fila in lector:
                id, nombre, edad, saturacion, vacunas, prueba_covid_str = fila

                id = int(id)
                edad = int(edad)
                saturacion = int(saturacion)
                vacunas = int(vacunas)

                if prueba_covid_str == "True":
                    prueba_covid = True
                elif prueba_covid_str == "False":
                    prueba_covid = False
                else:
                    prueba_covid = None

                paciente = Paciente(id, nombre, edad, saturacion, vacunas, prueba_covid)
                pacientes.append(paciente)

        return pacientes

    @staticmethod
    def escribir_archivo_resultados(pacientes, nombre_archivo):
        with open(nombre_archivo, 'w', newline='') as archivo:
            escritor = csv.writer(archivo)
            escritor.writerow(['ID', 'Nombre', 'Edad', 'Saturacion', 'Vacunas',
                              'Prueba_COVID', 'Destino'])

            for paciente in pacientes:
                prueba_covid_str = "" if paciente.prueba_covid is None else str(paciente.prueba_covid)
                escritor.writerow([paciente.id, paciente.nombre, paciente.edad,
                                  paciente.saturacion, paciente.vacunas,
                                  prueba_covid_str, paciente.destino])


class SistemaTriage:

    def __init__(self, max_camillas=10):
        self.max_camillas = max_camillas
        self.camillas_ocupadas = 0

    def asignar_destinos(self, pacientes):
        for paciente in pacientes:
            camillas_disponibles = self.max_camillas - self.camillas_ocupadas
            destino = paciente.determinar_destino(camillas_disponibles)


            if destino == "Urgencias":
                self.camillas_ocupadas += 1

    def generar_estadisticas(self, pacientes):
        total_pacientes = len(pacientes)
        sala_covid = sum(1 for p in pacientes if p.destino == "Sala COVID")
        urgencias = sum(1 for p in pacientes if p.destino == "Urgencias")
        espera = sum(1 for p in pacientes if p.destino == "Espera")

        estadisticas = {
            "Total de pacientes": total_pacientes,
            "Pacientes en Sala COVID": sala_covid,
            "Pacientes en Urgencias": urgencias,
            "Pacientes en Espera": espera,
            "Porcentaje Sala COVID": f"{(sala_covid/total_pacientes)*100:.2f}%",
            "Porcentaje Urgencias": f"{(urgencias/total_pacientes)*100:.2f}%",
            "Porcentaje Espera": f"{(espera/total_pacientes)*100:.2f}%"
        }

        return estadisticas

    @staticmethod
    def escribir_estadisticas(estadisticas, nombre_archivo):
        with open(nombre_archivo, 'w') as archivo:
            archivo.write("ESTADÍSTICAS DE CLASIFICACIÓN DE PACIENTES\n")
            archivo.write("========================================\n\n")

            for clave, valor in estadisticas.items():
                archivo.write(f"{clave}: {valor}\n")


def main():

    archivo_entrada = "pacientes_datos.csv"
    archivo_resultados = "pacientes_resultados.csv"
    archivo_estadisticas = "pacientes_estadisticas.txt"


    generador = GeneradorDatos()
    gestor = GestorArchivos()
    sistema = SistemaTriage(max_camillas=10)


    print("Generando datos aleatorios para 50 pacientes...")
    pacientes = generador.generar_pacientes(50)


    print(f"Escribiendo datos en {archivo_entrada}...")
    gestor.escribir_archivo_entrada(pacientes, archivo_entrada)


    print(f"Leyendo datos de {archivo_entrada}...")
    pacientes_leidos = gestor.leer_archivo_entrada(archivo_entrada)


    print("Asignando destinos a los pacientes...")
    sistema.asignar_destinos(pacientes_leidos)


    print(f"Escribiendo resultados en {archivo_resultados}...")
    gestor.escribir_archivo_resultados(pacientes_leidos, archivo_resultados)


    print("Generando estadísticas...")
    estadisticas = sistema.generar_estadisticas(pacientes_leidos)
    sistema.escribir_estadisticas(estadisticas, archivo_estadisticas)

    print("Proceso completado.")


    print("\nResumen de estadísticas:")
    for clave, valor in estadisticas.items():
        print(f"{clave}: {valor}")


if __name__ == "__main__":
    main()

Generando datos aleatorios para 50 pacientes...
Escribiendo datos en pacientes_datos.csv...
Leyendo datos de pacientes_datos.csv...
Asignando destinos a los pacientes...
Escribiendo resultados en pacientes_resultados.csv...
Generando estadísticas...
Proceso completado.

Resumen de estadísticas:
Total de pacientes: 50
Pacientes en Sala COVID: 36
Pacientes en Urgencias: 10
Pacientes en Espera: 4
Porcentaje Sala COVID: 72.00%
Porcentaje Urgencias: 20.00%
Porcentaje Espera: 8.00%


##Ejercicio 2

In [8]:
import random
import csv
from datetime import datetime

class Persona:

    def __init__(self, id, nombre, edad, tiene_tarjetas, reportado, tipo_contrato,
                 credito_otra_entidad, credito_misma_entidad):
        self.id = id
        self.nombre = nombre
        self.edad = edad
        self.tiene_tarjetas = tiene_tarjetas
        self.reportado = reportado
        self.tipo_contrato = tipo_contrato
        self.credito_otra_entidad = credito_otra_entidad
        self.credito_misma_entidad = credito_misma_entidad
        self.puntuacion = 0
        self.aprobado = False

    def evaluar_credito(self):
        self.puntuacion = 0


        if self.tiene_tarjetas:
            self.puntuacion += 1

        if not self.reportado:
            self.puntuacion += 3

        if self.tipo_contrato == "Definido":

            self.puntuacion += 2
        elif self.tipo_contrato == "Indefinido":

            self.puntuacion += 3


        if self.credito_otra_entidad:
            self.puntuacion += 2


        if self.credito_misma_entidad:
            self.puntuacion += 3


        if 18 <= self.edad <= 22:

            self.puntuacion += 2
        elif 23 <= self.edad <= 30:

            self.puntuacion += 4
        elif 31 <= self.edad <= 40:

            self.puntuacion += 3
        elif self.edad > 40:

            self.puntuacion += 1


        self.aprobado = self.puntuacion >= 6
        return self.aprobado

    def __str__(self):
        return (f"ID: {self.id}, Nombre: {self.nombre}, Edad: {self.edad}, "
                f"Puntuación: {self.puntuacion}, Aprobado: {self.aprobado}")


class GeneradorDatos:

    def __init__(self):
        self.nombres = ["Juan", "María", "Pedro", "Ana", "Carlos", "Laura", "Luis", "Sofía",
                       "Miguel", "Elena", "José", "Carmen", "Antonio", "Isabel", "David",
                       "Patricia", "Fernando", "Silvia", "Ricardo", "Mónica"]
        self.apellidos = ["García", "López", "Martínez", "Rodríguez", "González", "Fernández",
                         "Pérez", "Sánchez", "Ramírez", "Torres", "Díaz", "Vásquez", "Romero",
                         "Herrera", "Morales", "Ortega", "Delgado", "Castro", "Vargas", "Reyes"]

    def generar_nombre(self):
        nombre = random.choice(self.nombres)
        apellido = random.choice(self.apellidos)
        return f"{nombre} {apellido}"

    def generar_personas(self, cantidad):
        personas = []
        for i in range(1, cantidad + 1):
            nombre = self.generar_nombre()
            edad = random.randint(18, 65)


            tiene_tarjetas = random.choice([True, False])
            reportado = random.choice([True, False])
            tipo_contrato = random.choice(["Definido", "Indefinido", "No trabaja"])
            credito_otra_entidad = random.choice([True, False])
            credito_misma_entidad = random.choice([True, False])

            persona = Persona(i, nombre, edad, tiene_tarjetas, reportado, tipo_contrato,
                             credito_otra_entidad, credito_misma_entidad)
            personas.append(persona)

        return personas


class GestorArchivos:

    @staticmethod
    def escribir_archivo_entrada(personas, nombre_archivo):
        with open(nombre_archivo, 'w', newline='') as archivo:
            escritor = csv.writer(archivo)
            escritor.writerow(['ID', 'Nombre', 'Edad', 'Tiene_Tarjetas',
                              'Reportado', 'Tipo_Contrato',
                              'Credito_Otra_Entidad', 'Credito_Misma_Entidad'])

            for persona in personas:
                escritor.writerow([persona.id, persona.nombre, persona.edad,
                                  persona.tiene_tarjetas, persona.reportado,
                                  persona.tipo_contrato, persona.credito_otra_entidad,
                                  persona.credito_misma_entidad])

    @staticmethod
    def leer_archivo_entrada(nombre_archivo):
        personas = []
        with open(nombre_archivo, 'r') as archivo:
            lector = csv.reader(archivo)
            next(lector)

            for fila in lector:
                id, nombre, edad, tiene_tarjetas, reportado, tipo_contrato, credito_otra_entidad, credito_misma_entidad = fila


                id = int(id)
                edad = int(edad)
                tiene_tarjetas = tiene_tarjetas.lower() == 'true'
                reportado = reportado.lower() == 'true'
                credito_otra_entidad = credito_otra_entidad.lower() == 'true'
                credito_misma_entidad = credito_misma_entidad.lower() == 'true'

                persona = Persona(id, nombre, edad, tiene_tarjetas, reportado,
                                 tipo_contrato, credito_otra_entidad, credito_misma_entidad)
                personas.append(persona)

        return personas

    @staticmethod
    def escribir_archivo_resultados(personas, nombre_archivo):
        with open(nombre_archivo, 'w', newline='') as archivo:
            escritor = csv.writer(archivo)
            escritor.writerow(['ID', 'Nombre', 'Edad', 'Tiene_Tarjetas',
                              'Reportado', 'Tipo_Contrato',
                              'Credito_Otra_Entidad', 'Credito_Misma_Entidad',
                              'Puntuacion', 'Credito_Aprobado'])

            for persona in personas:
                escritor.writerow([persona.id, persona.nombre, persona.edad,
                                  persona.tiene_tarjetas, persona.reportado,
                                  persona.tipo_contrato, persona.credito_otra_entidad,
                                  persona.credito_misma_entidad, persona.puntuacion,
                                  persona.aprobado])


class SistemaCredito:

    def procesar_solicitudes(self, personas):

        for persona in personas:
            persona.evaluar_credito()

    def generar_estadisticas(self, personas):

        total_personas = len(personas)
        aprobados = sum(1 for p in personas if p.aprobado)
        rechazados = total_personas - aprobados
        puntuacion_promedio = sum(p.puntuacion for p in personas) / total_personas


        aprobados_18_22 = sum(1 for p in personas if p.aprobado and 18 <= p.edad <= 22)
        aprobados_23_30 = sum(1 for p in personas if p.aprobado and 23 <= p.edad <= 30)
        aprobados_31_40 = sum(1 for p in personas if p.aprobado and 31 <= p.edad <= 40)
        aprobados_mayor_40 = sum(1 for p in personas if p.aprobado and p.edad > 40)


        rechazados_reportados = sum(1 for p in personas if not p.aprobado and p.reportado)

        estadisticas = {
            "Total de solicitudes": total_personas,
            "Créditos aprobados": aprobados,
            "Créditos rechazados": rechazados,
            "Porcentaje de aprobación": f"{(aprobados/total_personas)*100:.2f}%",
            "Puntuación promedio": f"{puntuacion_promedio:.2f}",
            "Aprobados entre 18-22 años": aprobados_18_22,
            "Aprobados entre 23-30 años": aprobados_23_30,
            "Aprobados entre 31-40 años": aprobados_31_40,
            "Aprobados mayores de 40 años": aprobados_mayor_40,
            "Rechazados por estar reportados": rechazados_reportados
        }

        return estadisticas

    @staticmethod
    def escribir_estadisticas(estadisticas, nombre_archivo):

        with open(nombre_archivo, 'w') as archivo:
            archivo.write("ESTADÍSTICAS DE EVALUACIÓN DE CRÉDITOS\n")
            archivo.write("=====================================\n\n")

            for clave, valor in estadisticas.items():
                archivo.write(f"{clave}: {valor}\n")


def main():

    archivo_entrada = "solicitantes_datos.csv"
    archivo_resultados = "solicitantes_resultados.csv"
    archivo_estadisticas = "creditos_estadisticas.txt"


    try:
        m = int(input("Ingrese la cantidad de personas a evaluar: "))
        if m <= 0:
            print("La cantidad debe ser un número positivo. Se usará 50 por defecto.")
            m = 50
    except ValueError:
        print("Entrada inválida. Se usará 50 por defecto.")
        m = 50


    generador = GeneradorDatos()
    gestor = GestorArchivos()
    sistema = SistemaCredito()


    print(f"Generando datos aleatorios para {m} personas...")
    personas = generador.generar_personas(m)


    print(f"Escribiendo datos en {archivo_entrada}...")
    gestor.escribir_archivo_entrada(personas, archivo_entrada)


    print(f"Leyendo datos de {archivo_entrada}...")
    personas_leidas = gestor.leer_archivo_entrada(archivo_entrada)


    print("Evaluando solicitudes de crédito...")
    sistema.procesar_solicitudes(personas_leidas)


    print(f"Escribiendo resultados en {archivo_resultados}...")
    gestor.escribir_archivo_resultados(personas_leidas, archivo_resultados)


    print("Generando estadísticas...")
    estadisticas = sistema.generar_estadisticas(personas_leidas)
    sistema.escribir_estadisticas(estadisticas, archivo_estadisticas)

    print("Proceso completado.")


    aprobados = sum(1 for p in personas_leidas if p.aprobado)
    rechazados = m - aprobados
    print(f"\nResumen de resultados para {m} solicitudes:")
    print(f"Créditos aprobados: {aprobados} ({(aprobados/m)*100:.2f}%)")
    print(f"Créditos rechazados: {rechazados} ({(rechazados/m)*100:.2f}%)")


    print("\nEjemplos de solicitantes evaluados:")
    for i, persona in enumerate(personas_leidas[:5], 1):
        estado = "APROBADO" if persona.aprobado else "RECHAZADO"
        print(f"{i}. {persona.nombre} (ID: {persona.id}) - Puntuación: {persona.puntuacion} - {estado}")


if __name__ == "__main__":
    main()

Ingrese la cantidad de personas a evaluar: 9
Generando datos aleatorios para 9 personas...
Escribiendo datos en solicitantes_datos.csv...
Leyendo datos de solicitantes_datos.csv...
Evaluando solicitudes de crédito...
Escribiendo resultados en solicitantes_resultados.csv...
Generando estadísticas...
Proceso completado.

Resumen de resultados para 9 solicitudes:
Créditos aprobados: 6 (66.67%)
Créditos rechazados: 3 (33.33%)

Ejemplos de solicitantes evaluados:
1. Ana Reyes (ID: 1) - Puntuación: 9 - APROBADO
2. Fernando Pérez (ID: 2) - Puntuación: 4 - RECHAZADO
3. Elena López (ID: 3) - Puntuación: 4 - RECHAZADO
4. Laura Torres (ID: 4) - Puntuación: 9 - APROBADO
5. Luis Torres (ID: 5) - Puntuación: 5 - RECHAZADO


##Ejercicio 3

In [7]:
import random
import csv
from datetime import datetime
import matplotlib.pyplot as plt

class SensorHumedad:

    def __init__(self, id, ubicacion):

        self.id = id
        self.ubicacion = ubicacion
        self.registros = []

    def obtener_lectura(self):
        return round(random.uniform(0, 10), 1)

    def registrar_lecturas(self, cantidad):
        for i in range(cantidad):
            timestamp = datetime.now()
            valor = self.obtener_lectura()
            self.registros.append({
                'timestamp': timestamp,
                'valor': valor
            })

    def obtener_registros(self):
        return self.registros


class SistemaAlerta:

    def __init__(self):
        self.alertas = []

    def evaluar_humedad(self, timestamp, valor):
        alerta = None

        if valor < 3:
            alerta = {
                'timestamp': timestamp,
                'valor': valor,
                'tipo': 'BAJA',
                'mensaje': 'ALERTA: Humedad baja detectada'
            }
        elif valor > 8:
            alerta = {
                'timestamp': timestamp,
                'valor': valor,
                'tipo': 'ALTA',
                'mensaje': 'ALERTA: Humedad alta detectada'
            }

        if alerta:
            self.alertas.append(alerta)
            return alerta

        return None

    def obtener_alertas(self):
        return self.alertas


class Actuador:

    def __init__(self):
        self.grifo_abierto = False
        self.sistema_absorcion_activo = False
        self.acciones = []

    def evaluar_condiciones(self, timestamp, valor):
        accion = {
            'timestamp': timestamp,
            'valor': valor,
            'accion_grifo': None,
            'accion_absorcion': None,
            'estado_grifo': self.grifo_abierto,
            'estado_absorcion': self.sistema_absorcion_activo
        }


        if valor < 3:
            if not self.grifo_abierto:
                self.grifo_abierto = True
                accion['accion_grifo'] = 'ABRIR'

        elif 3 <= valor <= 8:
            if self.grifo_abierto:
                self.grifo_abierto = False
                accion['accion_grifo'] = 'CERRAR'


        elif valor > 8:
            if self.grifo_abierto:
                self.grifo_abierto = False
                accion['accion_grifo'] = 'CERRAR'

            if not self.sistema_absorcion_activo:
                self.sistema_absorcion_activo = True
                accion['accion_absorcion'] = 'ACTIVAR'


        if valor <= 8 and self.sistema_absorcion_activo:
            self.sistema_absorcion_activo = False
            accion['accion_absorcion'] = 'DESACTIVAR'


        accion['estado_grifo'] = self.grifo_abierto
        accion['estado_absorcion'] = self.sistema_absorcion_activo

        self.acciones.append(accion)
        return accion

    def obtener_acciones(self):
        return self.acciones


class SistemaRiego:

    def __init__(self, id_sensor, ubicacion):
        self.sensor = SensorHumedad(id_sensor, ubicacion)
        self.sistema_alerta = SistemaAlerta()
        self.actuador = Actuador()
        self.registros_completos = []

    def ejecutar_simulacion(self, num_lecturas=20):
        self.sensor.registrar_lecturas(num_lecturas)

        for registro in self.sensor.obtener_registros():
            timestamp = registro['timestamp']
            valor = registro['valor']


            alerta = self.sistema_alerta.evaluar_humedad(timestamp, valor)


            accion = self.actuador.evaluar_condiciones(timestamp, valor)


            registro_completo = {
                'timestamp': timestamp,
                'valor': valor,
                'alerta': alerta['tipo'] if alerta else None,
                'grifo': accion['estado_grifo'],
                'absorcion': accion['estado_absorcion'],
                'accion_grifo': accion['accion_grifo'],
                'accion_absorcion': accion['accion_absorcion']
            }

            self.registros_completos.append(registro_completo)

    def generar_reporte(self):
        total_registros = len(self.registros_completos)
        alertas_bajas = sum(1 for r in self.registros_completos if r['alerta'] == 'BAJA')
        alertas_altas = sum(1 for r in self.registros_completos if r['alerta'] == 'ALTA')
        veces_grifo_abierto = sum(1 for r in self.registros_completos if r['accion_grifo'] == 'ABRIR')
        veces_grifo_cerrado = sum(1 for r in self.registros_completos if r['accion_grifo'] == 'CERRAR')
        veces_absorcion_activada = sum(1 for r in self.registros_completos if r['accion_absorcion'] == 'ACTIVAR')

        valores_humedad = [r['valor'] for r in self.registros_completos]
        humedad_promedio = sum(valores_humedad) / total_registros if total_registros > 0 else 0

        return {
            'total_registros': total_registros,
            'alertas_baja_humedad': alertas_bajas,
            'alertas_alta_humedad': alertas_altas,
            'veces_grifo_abierto': veces_grifo_abierto,
            'veces_grifo_cerrado': veces_grifo_cerrado,
            'veces_absorcion_activada': veces_absorcion_activada,
            'humedad_promedio': round(humedad_promedio, 2),
            'humedad_minima': round(min(valores_humedad), 2) if valores_humedad else 0,
            'humedad_maxima': round(max(valores_humedad), 2) if valores_humedad else 0
        }


class GestorArchivos:

    @staticmethod
    def escribir_registros(registros, nombre_archivo):
        with open(nombre_archivo, 'w', newline='') as archivo:
            escritor = csv.writer(archivo)
            escritor.writerow(['Timestamp', 'Valor', 'Alerta', 'Grifo', 'Absorcion',
                              'Accion_Grifo', 'Accion_Absorcion'])

            for registro in registros:
                escritor.writerow([
                    registro['timestamp'],
                    registro['valor'],
                    registro['alerta'] if registro['alerta'] else '',
                    'Abierto' if registro['grifo'] else 'Cerrado',
                    'Activo' if registro['absorcion'] else 'Inactivo',
                    registro['accion_grifo'] if registro['accion_grifo'] else '',
                    registro['accion_absorcion'] if registro['accion_absorcion'] else ''
                ])

    @staticmethod
    def escribir_reporte(reporte, nombre_archivo):
        with open(nombre_archivo, 'w') as archivo:
            archivo.write("REPORTE DEL SISTEMA DE RIEGO\n")
            archivo.write("===========================\n\n")

            for clave, valor in reporte.items():
                archivo.write(f"{clave.replace('_', ' ').title()}: {valor}\n")


class VisualizadorDatos:

    @staticmethod
    def generar_grafico_humedad(registros, nombre_archivo):
        valores = [r['valor'] for r in registros]
        timestamps = [i for i in range(len(registros))]

        plt.figure(figsize=(10, 6))

        plt.axhspan(0, 3, alpha=0.2, color='red', label='Zona de Baja Humedad')
        plt.axhspan(8, 10, alpha=0.2, color='blue', label='Zona de Alta Humedad')
        plt.axhspan(3, 8, alpha=0.2, color='green', label='Zona Ideal')


        plt.plot(timestamps, valores, 'o-', color='black', label='Humedad')

        plt.axhline(y=3, color='r', linestyle='--')
        plt.axhline(y=8, color='b', linestyle='--')

        plt.title('Registro de Humedad del Suelo')
        plt.xlabel('Número de Medición')
        plt.ylabel('Nivel de Humedad (0-10)')
        plt.legend()
        plt.grid(True)

        plt.savefig(nombre_archivo)
        plt.close()


def main():
    archivo_registros = "registros_humedad.csv"
    archivo_reporte = "reporte_sistema.txt"
    archivo_grafico = "grafico_humedad.png"

    sistema = SistemaRiego(1, "Jardín Principal")


    print("Ejecutando simulación de sistema de riego...")
    sistema.ejecutar_simulacion(20)


    registros = sistema.registros_completos

    print(f"Escribiendo registros en {archivo_registros}...")
    GestorArchivos.escribir_registros(registros, archivo_registros)

    print("Generando reporte estadístico...")
    reporte = sistema.generar_reporte()
    GestorArchivos.escribir_reporte(reporte, archivo_reporte)


    print(f"Generando gráfico de humedad en {archivo_grafico}...")
    VisualizadorDatos.generar_grafico_humedad(registros, archivo_grafico)

    print("Simulación completada.")


    print("\nMuestra de registros:")
    for i, registro in enumerate(registros[:5], 1):
        alerta_str = f"ALERTA: {registro['alerta']}" if registro['alerta'] else "Sin alerta"
        grifo_str = "Abierto" if registro['grifo'] else "Cerrado"
        absorcion_str = "Activa" if registro['absorcion'] else "Inactiva"
        print(f"{i}. Humedad: {registro['valor']}, {alerta_str}, Grifo: {grifo_str}, Absorción: {absorcion_str}")

    print("\nResumen del reporte:")
    print(f"Total de registros: {reporte['total_registros']}")
    print(f"Alertas de baja humedad: {reporte['alertas_baja_humedad']}")
    print(f"Alertas de alta humedad: {reporte['alertas_alta_humedad']}")
    print(f"Humedad promedio: {reporte['humedad_promedio']}")


if __name__ == "__main__":
    main()

Ejecutando simulación de sistema de riego...
Escribiendo registros en registros_humedad.csv...
Generando reporte estadístico...
Generando gráfico de humedad en grafico_humedad.png...
Simulación completada.

Muestra de registros:
1. Humedad: 6.3, Sin alerta, Grifo: Cerrado, Absorción: Inactiva
2. Humedad: 9.9, ALERTA: ALTA, Grifo: Cerrado, Absorción: Activa
3. Humedad: 4.7, Sin alerta, Grifo: Cerrado, Absorción: Inactiva
4. Humedad: 0.8, ALERTA: BAJA, Grifo: Abierto, Absorción: Inactiva
5. Humedad: 4.4, Sin alerta, Grifo: Cerrado, Absorción: Inactiva

Resumen del reporte:
Total de registros: 20
Alertas de baja humedad: 7
Alertas de alta humedad: 5
Humedad promedio: 4.7


##Ejercicio 4

In [11]:
import random
import csv
from datetime import datetime, timedelta
import matplotlib.pyplot as plt

class MedicionDolar:

    def __init__(self, id, hora, valor):
        self.id = id
        self.hora = hora
        self.valor = valor
        self.comunicado = None

    def __str__(self):
        return f"ID: {self.id}, Hora: {self.hora.strftime('%H:%M')}, Valor: ${self.valor}"


class SimuladorDolar:

    def __init__(self, valor_base=4000, rango_variacion=2000):
        self.valor_base = valor_base
        self.rango_variacion = rango_variacion
        self.mediciones = []

    def generar_mediciones(self, cantidad=20):
        hora_inicio = datetime.now().replace(hour=8, minute=0, second=0, microsecond=0)


        intervalo = timedelta(minutes=30)


        valor_actual = self.valor_base

        for i in range(1, cantidad + 1):
            hora = hora_inicio + intervalo * (i - 1)


            variacion = random.uniform(-self.rango_variacion/2, self.rango_variacion/2)


            nuevo_valor = round(valor_actual + variacion, 2)


            medicion = MedicionDolar(i, hora, nuevo_valor)


            if nuevo_valor > 5000:
                medicion.comunicado = f"ALERTA: El dólar ha subido por encima de los $5000, alcanzando ${nuevo_valor:.2f}."
            elif nuevo_valor < 3000:
                medicion.comunicado = f"ALERTA: El dólar ha bajado por debajo de los $3000, llegando a ${nuevo_valor:.2f}."

            self.mediciones.append(medicion)

            valor_actual = nuevo_valor

    def generar_comunicado_cierre(self):
        if not self.mediciones:
            return "No hay mediciones disponibles para generar un comunicado de cierre."


        ultima_medicion = self.mediciones[-1]


        tendencia = "estable"
        if len(self.mediciones) > 1:
            penultima_medicion = self.mediciones[-2]

            if ultima_medicion.valor > penultima_medicion.valor:
                tendencia = "al alza"
            elif ultima_medicion.valor < penultima_medicion.valor:
                tendencia = "a la baja"

        return f"COMUNICADO DE CIERRE: El precio del dólar cerró en ${ultima_medicion.valor:.2f}, {tendencia} respecto a la medición anterior."

    def obtener_mediciones(self):
        return self.mediciones


class GestorArchivos:

    @staticmethod
    def escribir_mediciones(mediciones, nombre_archivo):
        with open(nombre_archivo, 'w', newline='') as archivo:
            escritor = csv.writer(archivo)
            escritor.writerow(['ID', 'Hora', 'Valor', 'Comunicado'])

            for medicion in mediciones:
                escritor.writerow([
                    medicion.id,
                    medicion.hora.strftime('%H:%M'),
                    medicion.valor,
                    medicion.comunicado or ''
                ])

    @staticmethod
    def escribir_reporte(mediciones, comunicado_cierre, nombre_archivo):
        with open(nombre_archivo, 'w') as archivo:
            archivo.write("REPORTE DE COMPORTAMIENTO DEL DÓLAR\n")
            archivo.write("===================================\n\n")

            archivo.write("Mediciones realizadas:\n")
            archivo.write("-----------------------\n")
            for medicion in mediciones:
                archivo.write(f"{medicion}\n")
                if medicion.comunicado:
                    archivo.write(f"  - {medicion.comunicado}\n")

            archivo.write("\n")
            archivo.write(comunicado_cierre)


class VisualizadorDatos:

    @staticmethod
    def generar_grafico(mediciones, nombre_archivo):
        valores = [m.valor for m in mediciones]
        horas = [m.hora.strftime('%H:%M') for m in mediciones]

        plt.figure(figsize=(12, 6))


        plt.axhspan(0, 3000, alpha=0.2, color='red', label='Zona de Alerta Baja')
        plt.axhspan(5000, 7000, alpha=0.2, color='red', label='Zona de Alerta Alta')
        plt.axhspan(3000, 5000, alpha=0.2, color='green', label='Zona Normal')


        plt.plot(horas, valores, 'o-', color='blue', linewidth=2, label='Valor del Dólar')


        plt.axhline(y=3000, color='r', linestyle='--')
        plt.axhline(y=5000, color='r', linestyle='--')


        plt.title('Evolución del Valor del Dólar durante el Día')
        plt.xlabel('Hora')
        plt.ylabel('Valor (Pesos)')
        plt.grid(True)
        plt.legend()


        plt.xticks(rotation=45)


        plt.tight_layout()

        plt.savefig(nombre_archivo)
        plt.close()


class AnalizadorDolar:

    @staticmethod
    def calcular_estadisticas(mediciones):
        if not mediciones:
            return {}

        valores = [m.valor for m in mediciones]

        valor_minimo = min(valores)
        valor_maximo = max(valores)
        valor_promedio = sum(valores) / len(valores)


        suma_cuadrados = sum((x - valor_promedio) ** 2 for x in valores)
        desviacion_estandar = (suma_cuadrados / len(valores)) ** 0.5


        alertas_alta = sum(1 for m in mediciones if m.valor > 5000)
        alertas_baja = sum(1 for m in mediciones if m.valor < 3000)

        primer_valor = mediciones[0].valor
        ultimo_valor = mediciones[-1].valor
        tendencia = ultimo_valor - primer_valor

        return {
            'valor_minimo': round(valor_minimo, 2),
            'valor_maximo': round(valor_maximo, 2),
            'valor_promedio': round(valor_promedio, 2),
            'volatilidad': round(desviacion_estandar, 2),
            'alertas_alta': alertas_alta,
            'alertas_baja': alertas_baja,
            'tendencia_general': round(tendencia, 2)
        }

    @staticmethod
    def escribir_estadisticas(estadisticas, nombre_archivo):
        with open(nombre_archivo, 'w') as archivo:
            archivo.write("ESTADÍSTICAS DEL COMPORTAMIENTO DEL DÓLAR\n")
            archivo.write("=========================================\n\n")

            archivo.write(f"Valor mínimo: ${estadisticas['valor_minimo']}\n")
            archivo.write(f"Valor máximo: ${estadisticas['valor_maximo']}\n")
            archivo.write(f"Valor promedio: ${estadisticas['valor_promedio']}\n")
            archivo.write(f"Volatilidad (desviación estándar): ${estadisticas['volatilidad']}\n")
            archivo.write(f"Número de alertas por alta: {estadisticas['alertas_alta']}\n")
            archivo.write(f"Número de alertas por baja: {estadisticas['alertas_baja']}\n")

            tendencia = estadisticas['tendencia_general']
            if tendencia > 0:
                archivo.write(f"Tendencia general: Al alza (+${tendencia})\n")
            elif tendencia < 0:
                archivo.write(f"Tendencia general: A la baja (${tendencia})\n")
            else:
                archivo.write("Tendencia general: Estable\n")


def main():
    archivo_mediciones = "mediciones_dolar.csv"
    archivo_reporte = "reporte_dolar.txt"
    archivo_grafico = "grafico_dolar.png"
    archivo_estadisticas = "estadisticas_dolar.txt"


    simulador = SimuladorDolar(valor_base=4000, rango_variacion=2000)


    print("Generando mediciones aleatorias del dólar...")
    simulador.generar_mediciones(20)


    mediciones = simulador.obtener_mediciones()


    print("Generando comunicado de cierre...")
    comunicado_cierre = simulador.generar_comunicado_cierre()


    print(f"Escribiendo mediciones en {archivo_mediciones}...")
    GestorArchivos.escribir_mediciones(mediciones, archivo_mediciones)


    print(f"Escribiendo reporte en {archivo_reporte}...")
    GestorArchivos.escribir_reporte(mediciones, comunicado_cierre, archivo_reporte)


    print("Generando estadísticas...")
    estadisticas = AnalizadorDolar.calcular_estadisticas(mediciones)
    AnalizadorDolar.escribir_estadisticas(estadisticas, archivo_estadisticas)


    print(f"Generando gráfico en {archivo_grafico}...")
    VisualizadorDatos.generar_grafico(mediciones, archivo_grafico)

    print("Simulación completada.")


    print("\nPrimeras mediciones:")
    for medicion in mediciones[:5]:
        print(f"- {medicion.hora.strftime('%H:%M')}: ${medicion.valor:.2f}")
        if medicion.comunicado:
            print(f"  {medicion.comunicado}")


    print("\n" + comunicado_cierre)

    print("\nResumen de estadísticas:")
    print(f"Valor mínimo: ${estadisticas['valor_minimo']}")
    print(f"Valor máximo: ${estadisticas['valor_maximo']}")
    print(f"Valor promedio: ${estadisticas['valor_promedio']}")


if __name__ == "__main__":
    main()

Generando mediciones aleatorias del dólar...
Generando comunicado de cierre...
Escribiendo mediciones en mediciones_dolar.csv...
Escribiendo reporte en reporte_dolar.txt...
Generando estadísticas...
Generando gráfico en grafico_dolar.png...
Simulación completada.

Primeras mediciones:
- 08:00: $3520.51
- 08:30: $3484.01
- 09:00: $4131.01
- 09:30: $3587.90
- 10:00: $2784.62
  ALERTA: El dólar ha bajado por debajo de los $3000, llegando a $2784.62.

COMUNICADO DE CIERRE: El precio del dólar cerró en $3490.89, a la baja respecto a la medición anterior.

Resumen de estadísticas:
Valor mínimo: $2059.98
Valor máximo: $4131.01
Valor promedio: $2998.91
